In [1]:
import psycopg2


In [2]:
# split the file into with_link and no_link tables
import json
import os

input_directory = "/Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data"
output_directory = "/Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data/split data"

for filename in os.listdir(input_directory):
    if filename.startswith("issue_") and filename.endswith(".json"):
        input_filepath = os.path.join(input_directory, filename)
        output_filepath_no_links = os.path.join(output_directory, f"{filename}_no_links.json")
        output_filepath_with_links = os.path.join(output_directory, f"{filename}_with_links.json")

        with open(input_filepath, "r") as input_file:
            data = json.load(input_file)

        # Extract the "linkedCommits" and remove it from the original data
        linked_commits = data["record"].pop("linkedCommits", [])

        # Write the data without "linkedCommits" to a separate file
        with open(output_filepath_no_links, "w") as output_file:
            json.dump(data, output_file, indent=4)

        # Create a new dictionary with "linkedCommits" and "id" as primary key
        new_data = {
            "id": data["id"],
            "linkedCommits": linked_commits
        }

        # Write the data with "linkedCommits" and "id" to a separate file
        with open(output_filepath_with_links, "w") as output_file:
            json.dump(new_data, output_file, indent=4)


In [25]:
# insert into json_no_link table

import os
import json
import psycopg2

connection_params = {
    "host": "localhost",
    "port": "5432",
    "database": "issuesCheckpoint_Graphviz",
    "user": "postgres",
    "password": "Wuzhonghao0313!"
}

json_directory = "/Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data/split data"

conn = psycopg2.connect(**connection_params)
cur = conn.cursor()

try:
    for filename in os.listdir(json_directory):
        if filename.endswith("_no_links.json"):
            filepath = os.path.join(json_directory, filename)
            with open(filepath, "r") as file:
                data = json.load(file)

            id = int(filename.split("_")[1].split(".")[0])
            record = data["record"]

            # Extract the timestamp values and handle empty strings
            created = record.get("created") or None
            resolved = record.get("resolved") or None

            # Remove the 'Z' character from the timestamp strings
            created = created.rstrip('Z') if created else None
            resolved = resolved.rstrip('Z') if resolved else None

            query = """
            INSERT INTO json_no_links (id, url, summary, description, type, priority, status, resolution, created, resolved, labels, versions, fixVersions, comments)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, TO_TIMESTAMP(%s, 'YYYY-MM-DDTHH24:MI:SS.US'), TO_TIMESTAMP(%s, 'YYYY-MM-DDTHH24:MI:SS.US'), %s, %s, %s, %s)
            """
            values = (
                id,
                record.get("url"),
                record.get("summary"),
                record.get("description"),
                record.get("type"),
                record.get("priority"),
                record.get("status"),
                record.get("resolution"),
                created,
                resolved,
                json.dumps(record.get("labels")),
                json.dumps(record.get("versions")),
                json.dumps(record.get("fixVersions")),
                json.dumps(record.get("comments"))
            )

            cur.execute(query, values)

    conn.commit()
    print("Data inserted successfully!")

except psycopg2.Error as e:
    conn.rollback()
    print("Error occurred while inserting data:", e)

finally:
    cur.close()
    conn.close()


Error occurred while inserting data: duplicate key value violates unique constraint "json_no_links_pkey"
DETAIL:  Key (id)=(1976) already exists.



In [17]:
# split the with_link json files
import json
import os

def split_json_files(filename, output_directory):
    with open(filename, 'r') as file:
        data = json.load(file)

    # Iterate over the linked commits and create separate files
    for index, commit in enumerate(data['linkedCommits']):
        primary_id = data['id']
        linked_id = commit['id']

        # Create the output file name using both primary and linked IDs
        output_filename = f"{primary_id}_{linked_id}.json"
        output_filepath = os.path.join(output_directory, output_filename)

        # Write the linked commit data to a separate JSON file
        with open(output_filepath, 'w') as output_file:
            json.dump(commit, output_file, indent=4)

# Specify the input directory
input_directory = '/Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data/split data'
# Specify the output directory
output_directory = '/Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data/split data/split link commits'

# Iterate over the files in the input directory
for filename in os.listdir(input_directory):
    if filename.startswith("issue_") and filename.endswith("json_with_links.json"):
        # Construct the full file path
        file_path = os.path.join(input_directory, filename)
        # Split the JSON file and save the linked commits to the output directory
        split_json_files(file_path, output_directory)


In [28]:
import chardet
import json

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        return encoding

# Provide the path to your JSON file
json_file_path = '/Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data/split data/split link commits/14_2577.json'

# Detect the encoding
encoding = detect_encoding(json_file_path)

# Print the detected encoding
print(f"The encoding of the JSON file is: {encoding}")


The encoding of the JSON file is: ascii


In [33]:
# insert into json_with_link table, exclude versionTags and changes

connection_params = {
    "host": "localhost",
    "port": "5432",
    "database": "issuesCheckpoint_Graphviz",
    "user": "postgres",
    "password": "Wuzhonghao0313!"
}

json_directory = "/Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data/split data/split link commits"

conn = psycopg2.connect(**connection_params)
cur = conn.cursor()

try:
    for filename in os.listdir(json_directory):
            filepath = os.path.join(json_directory, filename)
            with open(filepath, "r", encoding="ascii", errors="ignore") as file:
                try:
                    data = json.load(file)
                except json.JSONDecodeError:
                    print(f"Error: Invalid JSON file: {filepath}")
                    continue

            # Split the filename by underscore
            ids = filename.split('_')
            # Extract the primary ID and linked ID
            primary_id = ids[0]

            # Extract the timestamp values and handle empty strings
            created = data.get("created") or None

            # Remove the 'Z' character from the timestamp strings
            created = created.rstrip('Z') if created else None

            query = """ INSERT INTO json_with_links (id, linked_id, summary, description, status, url, sha, created, merged, labels)
            VALUES (%s, %s, %s, %s, %s, %s, %s, TO_TIMESTAMP(%s, 'YYYY-MM-DDTHH24:MI:SS.US'),  %s, %s)
            """
            values = (
                primary_id,
                data.get("id"),
                data.get("summary"),
                data.get("description"),
                data.get("status"),
                data.get("url"),
                data.get("sha"),
                created,
                data.get("merged"),
                json.dumps(data.get("labels"))
            )

            cur.execute(query, values)

    conn.commit()
    print("Data inserted successfully!")

except psycopg2.Error as e:
    conn.rollback()
    print("Error occurred while inserting data:", e)

finally:
    cur.close()
    conn.close()


Error: Invalid JSON file: /Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data/split data/split link commits/.DS_Store
Data inserted successfully!


In [36]:
# insert into versionTags table, in total 3 columns
import json
import psycopg2
import os

connection_params = {
    "host": "localhost",
    "port": "5432",
    "database": "issuesCheckpoint_Graphviz",
    "user": "postgres",
    "password": "Wuzhonghao0313!"
}

json_directory = "/Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data/split data/split link commits"

conn = psycopg2.connect(**connection_params)
cur = conn.cursor()

try:
    for filename in os.listdir(json_directory):
        filepath = os.path.join(json_directory, filename)
        with open(filepath, "r", encoding="ascii", errors="ignore") as file:
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                print(f"Error: Invalid JSON file: {filepath}")
                continue

        # Extract versionTags and find the biggest versionTag
        version_tags = data.get("versionTags") or []
        biggest_version_tag = max(version_tags) if version_tags else None

        # Insert data into the new table for versionTags
        version_tags_query = """
        INSERT INTO versionTags (linked_id, versionTags, newest_version)
        VALUES (%s, %s, %s)
        """
        version_tags_values = (
            data.get("id"),
            json.dumps(version_tags),
            biggest_version_tag
        )

        cur.execute(version_tags_query, version_tags_values)

    conn.commit()
    print("Data inserted successfully!")

except psycopg2.Error as e:
    conn.rollback()
    print("Error occurred while inserting data:", e)

finally:
    cur.close()
    conn.close()


Error: Invalid JSON file: /Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data/split data/split link commits/.DS_Store
Data inserted successfully!


In [38]:
# insert into changes table
import psycopg2
import json
import os

connection_params = {
    "host": "localhost",
    "port": "5432",
    "database": "issuesCheckpoint_Graphviz",
    "user": "postgres",
    "password": "Wuzhonghao0313!"
}

json_directory = "/Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data/split data/split link commits"

conn = psycopg2.connect(**connection_params)
cur = conn.cursor()

try:
    for filename in os.listdir(json_directory):
        filepath = os.path.join(json_directory, filename)
        with open(filepath, "r", encoding="ascii", errors="ignore") as file:
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                print(f"Error: Invalid JSON file: {filepath}")
                continue

        changes = data.get("changes", [])
        for change in changes:
            query = """
            INSERT INTO changes (linked_id, before, after)
            VALUES (%s, %s, %s)
            """
            values = (
                data.get("id"),
                change.get("before"),
                change.get("after")
            )
            cur.execute(query, values)

    conn.commit()
    print("Data inserted successfully!")

except psycopg2.Error as e:
    conn.rollback()
    print("Error occurred while inserting data:", e)

finally:
    cur.close()
    conn.close()


Error: Invalid JSON file: /Users/wuzhonghao/Desktop/2023 Spring/2023 Summer/data/split data/split link commits/.DS_Store
Data inserted successfully!
